# Extract MFCC features
This notebook shows how to use pykaldi to extract MFCC features from a wav file.
It also serves as an example for numpy and scikit-learn integration.

We begin by importing all the necesary components from pykaldi, numpy and sklearn. If you haven't done so, you'll need to install them in your system. You can install numpy and scikit using __pip__ (i.e., `pip install numpy scikit-learn`). For installation of pykaldi, please follow the [instructions](https://github.com/pykaldi/pykaldi#installation)

In [1]:
from kaldi.feat.mfcc import Mfcc, MfccOptions
from kaldi.matrix import SubVector, SubMatrix
from kaldi.util.options import ParseOptions
from kaldi.util.table import SequentialWaveReader
from kaldi.util.table import MatrixWriter
from numpy import mean
from sklearn.preprocessing import scale

We create a test scp file with a dummy key and the location of our test wav file. Make sure the location of the test file matches with the location of your pykaldi installation.

In [2]:
with open("testfile.scp", "w") as outpt:
    outpt.write("TEST /pykaldi/tools/kaldi/src/feat/test_data/test.wav")

PyKaldi option parsing API is slightly different from the underlying Kaldi option parsing API. Command-line options for the main script are registered by calling type-specific registration methods that accept name, default value and help string arguments e.g. \lstinline{min-duration} in the example. The `parse_args` method of a PyKaldi `ParseOptions` instance returns a simple namespace object containing the parsed option values for the main script. Parsed values for other options are directly written into the appropriate fields of associated options instances, e.g. `mfcc_opts` in the example.

In [3]:
usage = """Extract MFCC features.
           Usage:  example.py [opts...] <rspec> <wspec>
        """

po = ParseOptions(usage)
po.register_float("min-duration", 0.0,
                  "minimum segment duration")
mfcc_opts = MfccOptions()
mfcc_opts.frame_opts.samp_freq = 8000
mfcc_opts.register(po)

opts = po.parse_args()

/home/szy/miniconda3/envs/myenv/lib/python3.9/site-packages/ipykernel_launcher.py -f /home/szy/.local/share/jupyter/runtime/kernel-f4413ecd-cbcb-402c-a597-efd51d26253d.json 


In typical Kaldi fashion, input/output tables are constructed with read/write specifiers, strings that describe how the data should be read/written.

In [5]:
rspec, wspec = "scp:testfile.scp", "ark,t:test_mfcc.ark"

Kaldi objects are first class objects in PyKaldi. This allows them to be created, instanciated and passed as arguments to other objects or to functions.

In [6]:
# Create MFCC object and obtain sample frequency
mfcc = Mfcc(mfcc_opts)
sf = mfcc_opts.frame_opts.samp_freq

PyKaldi table readers/writers implement the context manager interface, hence they do not need to be closed when they are used in a `with` statement. PyKaldi table writers also support a pseudo-dictionary interface for writing given key value pairs. Since PyKaldi matrices implement NumPy array interface, they can be passed to functions expecting Numpy array arguments, such as `mean` and `scale`, without explicit conversion. The NumPy arrays returned from functions can be easily converted back to Kaldi vector and matrix types by constructing new `SubVector` and `SubMatrix` objects which share the underlying memory buffers with the source arrays whenever possible, i.e. no data is copied unless necessary.

In [7]:
with SequentialWaveReader(rspec) as reader, \
             MatrixWriter(wspec) as writer:
            
    for key, wav in reader:
        if wav.duration < opts.min_duration:
            continue
                    
        assert(wav.samp_freq >= sf)
        assert(wav.samp_freq % sf == 0)

        print(">>> print(wav.sample_freq)")
        print(wav.samp_freq)
        print()
        
        s = wav.data()
        print(">>> print(s)")
        print(s)
        print()
        
        # downsample to sf [default=8kHz]
        s = s[:,::int(wav.samp_freq / sf)]

        # mix-down stereo to mono
        m = SubVector(mean(s, axis=0))

        # compute MFCC features
        f = mfcc.compute_features(m, sf, 1.0)

        # standardize features
        f = SubMatrix(scale(f))
        print(">>> print(f)")
        print(f)
        print()
        
        # write features to archive
        writer[key] = f

WARNING (ipykernel_launcher.py[5.5.950~1-085be]:EnsureObjectLoaded():util/kaldi-table-inl.h:310) Failed to open file /pykaldi/tools/kaldi/src/feat/test_data/test.wav
ERROR (ipykernel_launcher.py[5.5.950~1-085be]:Value():util/kaldi-table-inl.h:164) Failed to load object from /pykaldi/tools/kaldi/src/feat/test_data/test.wav (to suppress this error, add the permissive (p, ) option to the rspecifier.


RuntimeError: C++ exception: kaldi::KaldiFatalError